In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_community.document_loaders import PyPDFLoader, UnstructuredXMLLoader
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

# OpenAI 모델 초기화
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

def classify_question(query):
    # 프롬프트 생성
    prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""\
    You are an expert in horse racing. Your task is to classify the given user question into one of the following categories:
    
    - 경마정보: Questions about general information such as rules, betting methods, and terminology.
    - 경주일정: Questions about race schedules, dates, times, or locations. If the question mentions specific dates, races, or schedules, prioritize this category even if other details (e.g., horse performance) are included.
    - 우승마기록: Questions about winning horses and their records.
    - 경주마정보: Questions about specific horses, their participation counts, rankings, or performance metrics.

    If the question does not match any category, return "Unknown".
    """),
    HumanMessage(content=f"User Question: {query}\n\nClassify this question into one of the categories:")
])

    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    response = chain.invoke({})
    
    return response.strip('-')

vectorstore_paths = {
    "경마정보": r"C:\Workspace\DA36_mini4_ma\min\vectors_new\vs_race_guide",
    "경주일정": r"C:\Workspace\DA36_mini4_ma\min\vectors_new\vs_schedule",
    "우승마기록": r"C:\Workspace\DA36_mini4_ma\min\vectors_new\vs_winners",
    "경주마정보": r"C:\Workspace\DA36_mini4_ma\min\vectors_new\vs_horse_info"
}

def summarize_query(query):
    if len(query.split()) <= 20:
        return False, query  

    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are an assistant that summarizes questions into concise queries for search."),
        HumanMessage(content=f"Original question: {query}\n\nSummarize this into a concise query:")
    ])
    summarized_query = (prompt | model | StrOutputParser()).invoke({"query": query})
    print("⚠️질문이 20단어를 초과하여 요약되었습니다.")
    return True, summarized_query 

def rag_and_prompt(query):
    category = classify_question(query)
    
    vectorstore_path = vectorstore_paths[category]
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vector_store = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)
    retriever = vector_store.as_retriever()

    is_summarized, summarized_query = summarize_query(query)  # 요약 수행
    if is_summarized:
        print(f"- 기존 질문: {query}\n- 요약된 질문: {summarized_query}\n")
    else:
        print(f"질문: {query}\n")

    results = retriever.get_relevant_documents(summarized_query)
    
    retrieved_data = "\n".join([doc.page_content for doc in results])
    
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""\
        당신은 전문적이고 애교가 많은 경마 안내 챗봇입니다. 
        질문에 대해 검색된 정보를 바탕으로 아주 상세하고 재미있는 답변을 제공합니다.
        예시: 2024년 12월 21일 서울 경주 일정을 물어보면, 경주의 시간과, 최근 성적이 좋은 말의 정보 등을 알려줘야 합니다.
        """),
        HumanMessage(content=f"""\
        사용자의 질문에 context만을 이용해 답변해 주세요.
        질문: {query}
        context: {retrieved_data}
        """)
    ])
    
    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    response = chain.invoke({"query": query})
    return response

def ask_question(query):
    response = rag_and_prompt(query)
    return response

In [11]:
query="나 이번에 경마장을 처음 가는데, 배팅을 어떻게 하는지 모르겠어. 하나의 말에만 배팅해야 하는 건가? 아니면 두마리? 순위까지 맞혀야 하는 거야?"
print(ask_question(query))

질문: 나 이번에 경마장을 처음 가는데, 배팅을 어떻게 하는지 모르겠어. 하나의 말에만 배팅해야 하는 건가? 아니면 두마리? 순위까지 맞혀야 하는 거야?

경마장에서 배팅을 처음 하신다니 정말 흥미진진한 경험이 되실 거예요! 경마 배팅은 다양한 방식으로 즐길 수 있는데요, 초보자라면 단승식이나 연승식을 추천드려요. 단승식은 1등으로 도착할 말을 맞추는 방식이고, 연승식은 1, 2, 3등 안에 들어올 말 중 하나를 맞추는 방식이에요. 적중률이 높고 환급률도 괜찮아서 처음 시작하기에 좋답니다.

만약 조금 더 도전해보고 싶으시다면, 복승식이나 쌍승식도 고려해보세요. 복승식은 1등과 2등으로 들어올 말을 순서에 상관없이 맞추는 방식이고, 쌍승식은 순서대로 맞추는 방식이에요. 각각의 방식에 따라 배당률이 다르니, 어떤 방식이 자신에게 맞는지 고민해보세요. 그리고 마권을 구매할 때는 경주번호, 승식, 금액, 그리고 선택하고 싶은 말을 잘 선택하셔야 해요. 즐거운 경마 경험 되시길 바랄게요! 🏇💨


In [12]:
query="나 이번에 경마장을 처음 가는데, 배팅을 어떻게 하는지 모르겠어. 하나의 말에만 배팅해야 하는 건가? 아니면 두마리? 순위까지 맞혀야 하는 거야? 흠 어려워"
print(ask_question(query))

⚠️질문이 20단어를 초과하여 요약되었습니다.
- 기존 질문: 나 이번에 경마장을 처음 가는데, 배팅을 어떻게 하는지 모르겠어. 하나의 말에만 배팅해야 하는 건가? 아니면 두마리? 순위까지 맞혀야 하는 거야? 흠 어려워
- 요약된 질문: 경마장에서 배팅하는 방법과 규칙은 무엇인가요?

경마장에서 배팅을 처음 하신다니, 정말 흥미진진한 경험이 될 거예요! 경마 배팅은 다양한 방식으로 즐길 수 있는데요, 초보자라면 단승식이나 연승식을 추천드려요. 

- **단승식**: 1등으로 도착할 말을 맞추는 방식이에요. 예를 들어, 7번 말을 선택해서 그 말이 1등으로 들어오면 적중입니다.
- **연승식**: 1, 2, 3등 안에 들어올 말 중 하나를 맞추는 방식이에요. 출주마가 7두 이하인 경우에는 1, 2등 안에 들어와야 적중이 됩니다.

이 외에도 여러 마리의 말을 조합해서 맞추는 복승식, 쌍승식, 삼복승식 등 다양한 배팅 방식이 있어요. 각 방식마다 적중률과 환급률이 다르니, 처음에는 단순한 방식으로 시작해보세요. 그리고 배당률을 참고하면 다른 사람들이 어떤 말을 우승마로 예측하고 있는지도 알 수 있으니, 이를 참고하는 것도 좋은 방법입니다. 즐거운 경마 경험 되세요! 🏇💨


In [13]:
query="나 12월 22일에 경마장을 처음 가서 잘 모르는데, 여러 마리의 말한테 배팅할 수 있는 거야? 나 그 날 3시 이후에 가는데 말들 경주력을 분석해주라"
print(ask_question(query))

⚠️질문이 20단어를 초과하여 요약되었습니다.
- 기존 질문: 나 12월 22일에 경마장을 처음 가서 잘 모르는데, 여러 마리의 말한테 배팅할 수 있는 거야? 나 그 날 3시 이후에 가는데 말들 경주력을 분석해주라
- 요약된 질문: Can I bet on multiple horses at the racetrack on December 22, and can you analyze the horses' performance for races after 3 PM?

경마장에 처음 가신다니 정말 흥미진진한 경험이 되실 거예요! 12월 22일에 여러 마리의 말에 배팅할 수 있는지 궁금하시군요. 네, 경마에서는 여러 마리의 말에 배팅할 수 있습니다. 이를 '복수 배팅'이라고 하며, 다양한 배팅 방식이 있어 전략적으로 접근할 수 있습니다.

이제 12월 22일 3시 이후에 열리는 경주에 대해 분석해드릴게요. 10경주 혼4 1600 핸디캡 경주가 예정되어 있습니다. 이 경주에서 주목할 만한 말들을 살펴보면:

1. **호크프린스**: 최근 경주에서 꾸준히 상위권에 머물고 있으며, 11월 24일 경주에서 5위를 기록했습니다. 경주력 예상은 1:42.0으로, 좋은 성적을 기대할 수 있습니다.

2. **브리도시리스**: 11월 24일 경주에서 4위를 기록하며 좋은 성적을 보였습니다. 경주력 예상은 1:43.8로, 주목할 만한 말입니다.

3. **달금**: 10월 19일 경주에서 4위를 기록하며 안정적인 성적을 보이고 있습니다. 경주력 예상은 1:43.6입니다.

이 외에도 다른 말들의 최근 성적과 경주력을 참고하여 배팅 전략을 세우시면 좋을 것 같습니다. 경마는 예측이 어려운 스포츠이니, 다양한 정보를 참고하여 신중하게 배팅하시길 바랍니다. 즐거운 경마 관람 되세요! 🏇🎉


In [20]:
print(ask_question('나 경마장을 처음 가는데, 마권 구매 방법에 대해 알려줘. 그리고 어떤 말에 배팅해야 할지 모르겠는데 로또처럼 자동으로 배팅하는 건 안될까? 제발 알려줘~'))

⚠️질문이 20단어를 초과하여 요약되었습니다.
- 기존 질문: 나 경마장을 처음 가는데, 마권 구매 방법에 대해 알려줘. 그리고 어떤 말에 배팅해야 할지 모르겠는데 로또처럼 자동으로 배팅하는 건 안될까? 제발 알려줘~
- 요약된 질문: 경마장에서 마권 구매 방법과 자동 배팅 옵션이 있는지 알려줘.

경마장에 처음 가신다니 정말 신나는 경험이 될 거예요! 마권 구매 방법에 대해 설명해 드릴게요. 경마는 다양한 승식으로 즐길 수 있는데, 초보자라면 단승식이나 연승식을 추천드려요. 단승식은 1등으로 도착할 말을 맞추는 방식이고, 연승식은 1, 2, 3등 안에 들어올 말을 맞추는 방식이에요.

마권을 구매하려면 먼저 경주번호를 선택하고, 승식을 선택한 후, 구입하고 싶은 금액을 정하면 됩니다. 예를 들어, 부산경남경마공원 1경주에서 단승식으로 7번 말을 2천 원에 구입할 수 있어요.

그리고 자동으로 배팅하는 방법도 있어요! '오토벳(Auto Bet)'이라는 기능을 이용하면 로또처럼 자동으로 배팅할 수 있답니다. 구매 시점 기준으로 단승식 저배당 5위 내에서 임의로 선택해 주는 방식이에요. 구매표에서 '마번' 대신 '자동(A)' 칸에 체크하면 됩니다.

말을 선택할 때는 예시장에서 말의 상태를 관찰하는 것도 좋은 방법이에요. 말의 눈빛, 털의 윤기, 땀의 양 등을 살펴보면 우승마를 예측하는 데 도움이 될 수 있답니다. 즐거운 경마 경험 되세요! 🐎🎉


In [21]:
query = "서울경마공원 12월 20일 오후 3시 이후 경마 경기 일정을 알려줘"
print(ask_question(query))

질문: 서울경마공원 12월 20일 오후 3시 이후 경마 경기 일정을 알려줘

2024년 12월 20일 서울경마공원에서 오후 3시 이후에 열리는 경마 경기는 다음과 같습니다:

- 5경주: 국 5등급, 1200M, 12두 출전, 출발 시각 15:40
- 6경주: 국 4등급, 1600M, 12두 출전, 출발 시각 16:35
- 7경주: 국 4등급, 1800M, 12두 출전, 출발 시각 17:30
- 8경주: 혼 3등급, 1200M, 12두 출전, 출발 시각 18:00

이 경주들에서는 다양한 등급과 거리를 가진 말들이 출전하여 흥미진진한 경주가 예상됩니다. 특히 7경주와 8경주는 각각 1800M와 1200M 거리로, 말들의 스피드와 지구력을 모두 볼 수 있는 좋은 기회가 될 것입니다. 즐거운 경마 관람 되세요! 🏇✨


In [24]:
query = "2024년 서울에서 가장 많이 우승한 말이 누구야?"
print(ask_question(query))

질문: 2024년 서울에서 가장 많이 우승한 말이 누구야?

2024년 서울에서 가장 많이 우승한 말은 "영광의월드"와 "크라운함성"입니다. 두 말 모두 6번의 우승을 기록했으며, "영광의월드"는 9번 출전하여 6승 1위, 1회 2위, 1회 3위를 기록하며 승률 66.7%를 자랑합니다. "크라운함성"은 8번 출전하여 6승 1위, 1회 3위를 기록하며 승률 75.0%를 기록했습니다. 두 말 모두 뛰어난 성적을 보여주며 서울 경마에서 두각을 나타냈습니다.
